In [ ]:
 !pip install -U langchain langchain-community langchain-core langchain-openai langchain-google-genai langchain-text-splitters chromadb faiss-cpu sentence-transformers pypdf pymupdf unstructured tiktoken huggingface-hub transformers accelerate bitsandbytes python-dotenv requests tqdm


In [ ]:
!pip install pymupdf

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxxxxxxxxxx"

In [ ]:
from langchain_community.document_loaders import TextLoader
speechdoc="/content/speech.txt"
loader = TextLoader(speechdoc)
text_document = loader.load()
text_document

In [ ]:
# web based loader
from langchain_community.document_loaders import WebBaseLoader
import bs4
from bs4 import SoupStrainer

## load,chunk and index the content of the html page

loader=WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
bs_kwargs=dict(parse_only=bs4.SoupStrainer(
    class_=("post-title","post-content","post-header"))
)
)

web_documents=loader.load()

In [ ]:
web_documents

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader
loader= PyMuPDFLoader("/content/attention.pdf")
pdf_documents=loader.load()

In [ ]:
pdf_documents

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,
                                             chunk_overlap=200)

documents=text_splitter.split_documents(web_documents)
documents

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings


#from langchain_community.embedding import OllamaEmbedding #this is for local machine only

In [ ]:
## Chroma vector database
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(
                            documents,
                            HuggingFaceEmbeddings()
                           )

In [ ]:
## FAISS vector detabase
from langchain_community.vectorstores import FAISS
db1 = FAISS.from_documents(documents[:15], HuggingFaceEmbeddings())

In [ ]:
query = "attention function can be discribed as mapping a query"
normal_search = db.similarity_search(query, k=1,)
normal_search

In [ ]:
print(normal_search)

In [ ]:
## manual embed query (similarity_search_by_vector)
embeddings=HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

db3=Chroma.from_documents(
                          documents,
                          embedding=embeddings,
                          collection_name="minilm_384"
                          )

query_embedding= embeddings.embed_query(query)

vector_search=db3.similarity_search_by_vector(query_embedding,k=1)
vector_search

In [ ]:
# pseudocode: not executed here, shows how retrieval + LLM combine
retrieved = db.similarity_search(query, k=3)
context = "\n\n".join([doc.page_content for doc in retrieved])
prompt = f"Answer using ONLY the context below:\n\n{context}\n\nQuestion: {query}\nAnswer:"
answer = llm.generate(prompt)   # llm could be OpenAI, local Llama, etc.
